In [2]:
import pandas as pd
import numpy as np

# Hierarchical indexing

### Tuple indexing

In [9]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [10]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [11]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### `MultiIndex`

In [13]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [14]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [15]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [16]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [17]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [18]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [19]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


### `MultiIndex` construction

In [26]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.157104  0.317687
  2  0.486794  0.408337
b 1  0.231609  0.436926
  2  0.106810  0.758586

In [27]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [28]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [29]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [30]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [31]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [32]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

An additional benefit of naming your `index`: if you `reset_index()`, your indexes will become columns, they will be already named with their names!

In [33]:
pop.reset_index()

,state,year,0
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [34]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  37.9  50.0  36.7  34.0  38.4
     2      38.0  37.5  39.0  37.3  50.0  37.3
2014 1      21.0  36.5  39.0  38.6  47.0  38.2
     2      30.0  36.1  51.0  37.2  39.0  36.5

Here we see where the multi-indexing for both rows and columns can come in very handy. This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number. With this in place we can, for example, index the top-level column by the person's name and get a full DataFrame containing just that person's information:

In [35]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      50.0  36.7
     2      39.0  37.3
2014 1      39.0  38.6
     2      51.0  37.2

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

In [36]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  37.9  50.0  36.7  34.0  38.4
     2      38.0  37.5  39.0  37.3  50.0  37.3
2014 1      21.0  36.5  39.0  38.6  47.0  38.2
     2      30.0  36.1  51.0  37.2  39.0  36.5

In [37]:
health_data['Guido', 'HR']

year  visit
2013  1        50.0
      2        39.0
2014  1        39.0
      2        51.0
Name: (Guido, HR), dtype: float64

In [38]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      36.0  37.9
     2      38.0  37.5

In [39]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        36.0
      2        38.0
2014  1        21.0
      2        30.0
Name: (Bob, HR), dtype: float64

>**Pro-tip**: If you're going to be working `MultiIndex` data, you'd need to dig a little deeper in the guts of indexing. For example, trying to do slicing with `:` inside a `tuple` will lead to a syntax error:

In [40]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-40-8e3cc151e316>, line 1)

>The way to go in this context is to use an `IndexSlice` object, which Pandas provides for precisely this situation. In will allow you to do arbitrary slices in any dimension using `:`. For example:

In [41]:
idx = pd.IndexSlice
# get info for the HR measurement on the first visit for all patients 
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,36.0,50.0,34.0
2014,1,21.0,39.0,47.0


# Rearranging indexed data

In [20]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.299371
      2      0.215429
c     1      0.648531
      2      0.164178
b     1      0.156052
      2      0.355852
dtype: float64

In [21]:
data['a':'b']

KeyError: 'Key length (1) was greater than MultiIndex lexsort depth (0)'

In [22]:
data = data.sort_index()
data

char  int
a     1      0.299371
      2      0.215429
b     1      0.156052
      2      0.355852
c     1      0.648531
      2      0.164178
dtype: float64

In [23]:
data['a':'b']

char  int
a     1      0.299371
      2      0.215429
b     1      0.156052
      2      0.355852
dtype: float64

### Stacking/unstacking

In [24]:
pop.unstack(level=0)

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [25]:
pop.unstack(level=1)

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


### Index setting/resetting

In [42]:
pop_flat = pop.reset_index()
pop_flat

,state,year,0
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [43]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [44]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [45]:
pop_flat.set_index('population')

,state,year
population,,
33871648,California,2000
37253956,California,2010
18976457,New York,2000
19378102,New York,2010
20851820,Texas,2000
25145561,Texas,2010


In [46]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      36.0  37.9  50.0  36.7  34.0  38.4
     2      38.0  37.5  39.0  37.3  50.0  37.3
2014 1      21.0  36.5  39.0  38.6  47.0  38.2
     2      30.0  36.1  51.0  37.2  39.0  36.5

In [47]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob       Guido         Sue       
type       HR  Temp    HR  Temp    HR   Temp
year                                        
2013     37.0  37.7  44.5  37.0  42.0  37.85
2014     25.5  36.3  45.0  37.9  43.0  37.35

In [48]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,41.166667,37.516667
2014,37.833333,37.183333
